In [77]:
import requests
import pandas as pd
import gzip
import io
import shutil
import os
import glob
from datetime import datetime


## Define date and url


In [44]:
index = 'BTCUSD'
type = 'spot_index'
base_url = 'https://public.bybit.com/'
date = '2019-10-01'
data_extension = '.csv.gz'
file_name = index + date + '_index_price' + data_extension
url = base_url + type + '/' + index + '/'  + file_name
compressed_path = 'data/raw/compressed/'
extracted_path = 'data/raw/extracted/'

In [45]:
https://public.bybit.com/spot_index/BTCUSD/BTCUSD2019-10-01_index_price.csv.gz

SyntaxError: invalid syntax (<ipython-input-45-ed6f835d45c1>, line 1)

## Get list of dates from the data source page

In [ ]:
url_dates = base_url + type + '/' + index + '/'

In [ ]:
r = requests.get(url_dates, allow_redirects=True)
dates_content = r.text
dates_content = dates_content.split('<li>')
dates_content = [row.split('"')[1] for row in dates_content]
dates_content = dates_content[1:]
dates = [row.split('BTCUSD')[1].split('_index')[0] for row in dates_content]

In [ ]:
df_dates = pd.DataFrame()
df_dates['date'] = dates
df_dates['date'] = pd.to_datetime(df_dates['date'])
df_dates['exists'] = 1

### Visualize the window the data is available

In [ ]:
import plotly.express as px

df = px.data.stocks()
fig = px.scatter(df_dates, x='date', y='exists')
fig.show()

## Get content from url

In [ ]:
def get_content(url):
    r = requests.get(url, allow_redirects=True)
    content = r.content
    return content

## Write the content into defined path

In [ ]:
def write_content(compressed_path, content, file_name):
    open(os.path.join(compressed_path,file_name), 'wb').write(content)

## Download and write content for each date

In [ ]:
def download_all_dates(*args):
    for date in dates:
        file_name = index + date + '_index_price' + data_extension
        url = base_url + type + '/' + index + '/'  + file_name
        content = get_content(url)
        write_content(compressed_path, content, file_name)

In [ ]:
args = {'dates' : dates,
        'index' : index,
        'data_extension' : data_extension,
        'base_url' : base_url,
        'file_name' : file_name,
        'compressed_path' : compressed_path}
download_all_dates(args)

## Unzip all files

In [46]:
def unzip_gzip(compressed_path, extracted_path, file_name):
    with gzip.open(os.path.join(compressed_path, file_name), 'rb') as f_in:
        file_name = file_name.replace('.gz','')
        with open(os.path.join(extracted_path,file_name), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [49]:
def unzip_files_in_path(*args):
    file_names = [file for file in os.listdir(compressed_path) if file.endswith(".gz")]
    for file_name in file_names:
        unzip_gzip(compressed_path, extracted_path, file_name)

In [50]:
args = {'dates' : dates,
        'index' : index,
        'data_extension' : data_extension,
        'base_url' : base_url,
        'compressed_path' : compressed_path}
unzip_files_in_path(args)

## Read extracted path and append data into one dataframe

In [64]:
def concat_csvs_to_dataframe(extracted_path):
    file_names = [file for file in os.listdir(extracted_path) if file.endswith(".csv")]#
    file_paths = [os.path.join(extracted_path, file_name) for file_name in file_names]
    data = pd.concat([pd.read_csv(file_path) for file_path in file_paths])
    return data

In [65]:
data = concat_csvs_to_dataframe(extracted_path)

## Save the concatenated dataframe

In [ ]:
def save_concat_data():
    concat_path = extracted_path.replace('extracted', 'concatenated')
    data.sort_values(by='start_at', ascending=True, inplace=True)
    start_date = datetime.fromtimestamp(data['start_at'].iloc[0]).date()
    end_date = datetime.fromtimestamp(data['start_at'].iloc[-1]).date()
    data_name = index + '_' + str(start_date) + '_' + str(end_date) + '_' + type + '.csv'

In [101]:
data.to_csv(os.path.join(concat_path,data_name))